This notebook demonstrates the analysis we made for polyribosome chains in our MemBrain v2 publication. The corresponding data is available on Zenodo: https://doi.org/10.5281/zenodo.4743933

What happened to get to this data?
1. MemBrain-pick prediction of ribosome positions on the nuclear envelope
2. Subtomogram averaging of ribosome positions, giving a RELION star file for each membrane
3. Analysis of data using MemBrain-stats with the following command:
```
membrain_stats geodesic_NN --in-folder $obj_folder  \
    --out-folder $out_folder \
    --num-neighbors 3 \
    --pixel-size-multiplier 7.84 \
    --pixel-size-multiplier-positions 7.84\
```

In [14]:
import napari
import numpy as np
import starfile
import trimesh
import os
from membrain_seg.segmentation.dataloading.data_utils import load_tomogram
import eulerangles
from skimage import measure

In [7]:
!curl https://zenodo.org/records/15050335/files/ribosome_stats_example.zip -O ribosome_stats_example.zip
!unzip ribosome_stats_example.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  462M  100  462M    0     0  2751k      0  0:02:52  0:02:52 --:--:-- 3161k 0     0  2572k      0  0:03:04  0:00:32  0:02:32 3696k 0     0  2721k      0  0:02:53  0:00:40  0:02:13 3317k   0  2841k      0  0:02:46  0:00:51  0:01:55 3460k5M    0     0  2793k      0  0:02:49  0:01:33  0:01:16 2846k  0     0  2733k      0  0:02:53  0:02:08  0:00:45 2773k6k      0  0:02:53  0:02:14  0:00:39 2624k
curl: (6) Could not resolve host: ribosome_stats_example.zip


In [15]:
data_folder="./ribosome_stats_example/data/"
stats_folder = "./ribosome_stats_example/stats/"
structure_file = ".ribosome_stats_example/CytoER_14.mrc"

In [16]:
viewer = napari.Viewer(ndisplay=3)

In [1]:
from example_functions import collect_between_angles

In [18]:
all_nn1_distances = []
all_nn2_distances = []
all_nn3_distances = []

all_nn1_angles = []
all_nn2_angles = []
all_nn3_angles = []

count = 0
prev_filename = None
for filename in os.listdir(data_folder):
    if not filename.endswith("obj"):
        continue
    count += 1
    prev_filename = filename
    print(count, filename)
    star_file = filename.replace(".obj", "_clusters.star")
    star_file = os.path.join(data_folder, star_file)
    mesh_file = os.path.join(data_folder, filename)
    stats_file = os.path.join(stats_folder, filename.replace(".obj", "_nearest_neighbors.star"))
    mesh = trimesh.load(mesh_file)
    star_data = starfile.read(star_file)
    nn_stats = starfile.read(stats_file)
    all_nn1_distances, all_nn2_distances, all_nn3_distances, \
        all_nn1_angles, all_nn2_angles, all_nn3_angles = collect_between_angles(
        nn_stats, 
        all_nn1_distances, 
        all_nn2_distances, 
        all_nn3_distances, 
        all_nn1_angles, 
        all_nn2_angles, 
        all_nn3_angles
    )

# gt positions, distances, and angles for the last file to display in Napari
start_positions = nn_stats[["start_positionX", "start_positionY", "start_positionZ"]].values / 7.82
nn1_pos = nn_stats[["nn0_positionX", "nn0_positionY", "nn0_positionZ"]].values / 7.82
nn2_pos = nn_stats[["nn1_positionX", "nn1_positionY", "nn1_positionZ"]].values / 7.82
nn3_pos = nn_stats[["nn2_positionX", "nn2_positionY", "nn2_positionZ"]].values / 7.82

distances = nn_stats["nn2_distance"].values

between_angles1 = nn_stats["nn0_angle"].values
between_angles2 = nn_stats["nn1_angle"].values
between_angles3 = nn_stats["nn2_angle"].values


1 1860.obj


2 2414.obj
3 1900.obj
4 2207.obj
5 2415.obj
6 2171.obj
7 2173.obj
8 2588.obj
9 2984.obj
10 2238.obj
11 1902.obj
12 2833.obj
13 2837.obj
14 2638.obj
15 2758.obj
16 2016.obj
17 2175.obj
18 1859.obj
19 2363.obj
20 2983.obj
21 2015.obj
22 2014.obj
23 2438.obj
24 2174.obj
25 1793.obj
26 2072.obj
27 2890.obj
28 1828.obj
29 2299.obj
30 1747.obj
31 2930.obj
32 2688.obj
33 2897.obj
34 1794.obj
35 1838.obj
36 2869.obj
37 2739.obj
38 1595.obj
39 2275.obj
40 2659.obj
41 2881.obj
42 1765.obj
43 2090.obj
44 2657.obj
45 2455.obj
46 2247.obj
47 973.obj
48 2093.obj
49 2865.obj
50 2859.obj
51 1763.obj
52 2915.obj
53 2069.obj
54 1831.obj
55 2647.obj
56 2653.obj
57 2860.obj
58 2876.obj
59 2686.obj
60 2679.obj
61 1364.obj
62 1576.obj
63 2297.obj
64 2452.obj
65 2308.obj
66 1832.obj
67 2644.obj
68 2151.obj
69 1699.obj
70 2390.obj
71 1882.obj
72 2408.obj
73 2805.obj
74 2797.obj
75 2796.obj
76 2351.obj
77 2437.obj
78 2392.obj
79 2802.obj
80 2631.obj
81 2143.obj
82 2355.obj
83 2209.obj
84 2792.obj
85 2988.obj
8

In [2]:
%load_ext autoreload
%autoreload 2

from example_functions import plot_orientations

plot_orientations(all_nn1_distances, all_nn2_distances, all_nn3_distances, all_nn1_angles, all_nn2_angles, all_nn3_angles)



NameError: name 'all_nn1_distances' is not defined

In [206]:

viewer.add_surface((mesh.vertices, mesh.faces[:, [2,1,0]]))
pos = star_data[["rlnCoordinateX", "rlnCoordinateY", "rlnCoordinateZ"]].values
ori = star_data[["rlnAngleRot", "rlnAngleTilt", "rlnAnglePsi"]].values

pos_layer = viewer.add_points(pos)
pos_layer.shading = "spherical"
struc = load_tomogram(structure_file).data * -1
struc[:, :, :35] = -10

verts, faces, normals, values = measure.marching_cubes(struc, 2, step_size=2)
verts -= (np.array(struc.shape) / 2)

rotation_matrices = eulerangles.euler2matrix(
    ori,
    axes="zyz",
    intrinsic=True,
    right_handed_rotation=False
)

/Users/lorenz.lamm/PhD_projects/MemBrain_stats/venv_membrain_stats/lib/python3.11/site-packages/mrcfile/mrcinterpreter.py:216: RuntimeWarning: Unrecognised machine stamp: 0x44 0x20 0x20 0x20
  warnings.warn(str(err), RuntimeWarning)


In [212]:
import numpy as np
from example_functions import iterative_chain_building, merge_chains, find_pos_index

# Define chain parameters
max_angle = 60 # angle between two orientations of ribosomes
max_angle_connection = 80 # angle between two connections of ribosomes
max_distance = 400 / 7.82 # maximum distance between two ribosomes in pixel space

positions = pos
nearest_neighbors = [
    [find_pos_index(curpos, start_positions) for curpos in [nn1, nn2, nn3]] for nn1, nn2, nn3 in zip(nn1_pos, nn2_pos, nn3_pos)
]
between_angles = [
    [between_angles1[k], between_angles2[k], between_angles3[k]] for k in range(len(between_angles1))
]

# Build chains iteratively
chains = iterative_chain_building(positions, nearest_neighbors, between_angles, max_angle, max_angle_connection, max_distance=max_distance)

merged_chains = chains
merged_chains = merge_chains(
    chains=chains,
    positions=positions,
    nearest_neighbors=nearest_neighbors,
    between_angles=between_angles,
    max_distance=400 / 7.82,
    max_angle_merge_orientation=max_angle,
    max_angle_merge_connection=max_angle_connection
)

30 before merge
30 after merge


In [213]:
all_lines = []
for idx, points in enumerate(merged_chains):
    points_array = positions[points]  # Convert list of points to a numpy array

    if len(points_array) > 1:
        lines = np.array([[points_array[i], points_array[i + 1]] for i in range(len(points_array) - 1)])
        all_lines.append(lines)
all_lines = np.concatenate(all_lines)
viewer.add_shapes(all_lines, shape_type='line', edge_color="black", edge_width=2.5, name="Lines")

<Shapes layer 'Lines' at 0x1ab025a90>

In [214]:
from example_functions import colormaps

surfaces = []
bigger_than2_chains = [chain for chain in merged_chains if len(chain) > 2]
idcs_not_in_chains = set(range(len(positions)))
for chain in bigger_than2_chains:
    for idx in chain:
        idcs_not_in_chains.remove(idx)

for nr, idx_chain in enumerate(bigger_than2_chains):
    chain_verts = []
    chain_faces = []
    for idx in idx_chain:
        cur_verts = verts.copy()
        rotmat = rotation_matrices[idx]
        subpos = pos[idx]
        cur_verts = np.dot(rotmat.T, cur_verts.T).T
        cur_verts += subpos
        chain_verts.append(cur_verts)
        chain_faces.append(faces.copy() + (len(chain_verts) - 1) * len(verts))
        # surfaces.append(viewer.add_surface((cur_verts,faces), colormap=colormaps[nr % len(colormaps)]))
    chain_verts = np.concatenate(chain_verts, axis=0)
    chain_faces = np.concatenate(chain_faces, axis=0)
    surfaces.append(viewer.add_surface((chain_verts,chain_faces), colormap=colormaps[nr % len(colormaps)]))

empty_chain_verts = []
empty_chain_faces = []
for idx in idcs_not_in_chains:
    cur_verts = verts.copy()
    rotmat = rotation_matrices[idx]
    subpos = pos[idx]
    cur_verts = np.dot(rotmat.T, cur_verts.T).T
    cur_verts += subpos
    empty_chain_verts.append(cur_verts)
    empty_chain_faces.append(faces.copy() + (len(empty_chain_verts) - 1) * len(verts))
    # surfaces.append(viewer.add_surface((cur_verts,faces), colormap="gray"))
empty_chain_verts = np.concatenate(empty_chain_verts, axis=0)
empty_chain_faces = np.concatenate(empty_chain_faces, axis=0)
surfaces.append(viewer.add_surface((empty_chain_verts,empty_chain_faces), colormap="gray"))